In [77]:
from google.cloud import bigquery

client = bigquery.Client(project="qwiklabs-gcp-04-14e3a67e50ad")
dataset_id = 'fraud_data_raw'

dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
dataset = client.create_dataset(dataset)  # Make an API request.

In [78]:
# ! bq load \
#   --source_format=CSV \
#   --autodetect \
#   --skip_leading_rows=1 \
#   fraud_data_raw.fraud_data_raw \
#   gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv

#Imported using bash shell

In [79]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
client

In [81]:
%%bigquery

SELECT * FROM `qwiklabs-gcp-04-14e3a67e50ad.fraud_data_raw.fraud_data_raw` LIMIT 1000


Query is running:   0%|          |

Downloading:   0%|          |

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0
5,281,32,Unemployed,25895,3,5411,False,NaT,False,1,202.173.84.210,Desktop,2024-04-30,1
6,300,30,Unemployed,0,4,2245,False,NaT,True,1,178.95.218.82,Tablet,2024-05-15,0
7,303,25,Self-Employed,36223,5,2409,False,NaT,True,1,202.173.163.25,Tablet,2024-03-24,0
8,330,35,Employed,42432,2,793,False,NaT,True,1,114.135.28.171,Tablet,2024-01-14,0
9,353,46,Self-Employed,0,3,8430,False,NaT,True,1,191.195.123.68,Desktop,2024-07-24,0


In [82]:
query_1 = """SELECT * FROM `qwiklabs-gcp-04-14e3a67e50ad.fraud_data_raw.fraud_data_raw`"""

fraud_df = client.query(query_1).to_dataframe()
fraud_df.head()

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0


In [83]:
print(fraud_df['Previous_Assistance_Date'].describe())

count          24854
unique           846
top       2023-09-08
freq              67
Name: Previous_Assistance_Date, dtype: object


## **Senity check for cols**

In [84]:
fraud_df["Employment_Status"].value_counts(dropna=False)

,count
Employment_Status,
Employed,16734
Self-Employed,16682
Unemployed,16584


In [85]:
# Frequency of each category

fraud_df["Device_Type"].value_counts(dropna=False)

,count
Device_Type,
Mobile,16733
Tablet,16652
Desktop,16615


In [86]:
fraud_df[["Employment_Status", "Device_Type"]].isna().sum()


,0
Employment_Status,0
Device_Type,0


**Hot Encoding**

In [87]:

cols_to_encode = ['Employment_Status', 'Device_Type']


df_encode = pd.get_dummies(
    fraud_df,
    columns = cols_to_encode,
    dtype=int  # ensures 0/1 integers instead of booleans
)

df_encode.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,0,0,1,0,1,0
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,0,1,0,0,0,1
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,0,1,0,0,1,0
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,0,0,1,1,0,0
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,1,0,0,0,1,0


In [88]:
print(len(fraud_df.columns))
print(len(df_encode.columns))

14
18


## **Creating Age bin and one-hot encoding them**

In [89]:

print("Step b: Binning Age field and one-hot encoding...")
age_bins = [0 ,18, 25, 35, 50, 65, 80]
age_labels = ["0-18", "18-24" ,"25-34", "35-50", "51-70", "70+"]
df_encode['Age_Bin'] = pd.cut(df_encode['Age'], bins=age_bins, labels=age_labels)

Step b: Binning Age field and one-hot encoding...


In [90]:
df_encode.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_Bin
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,0,0,1,0,1,0,51-70
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,0,1,0,0,0,1,51-70
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,0,1,0,0,1,0,25-34
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,0,0,1,1,0,0,25-34
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,1,0,0,0,1,0,35-50


## **One-hot encode the age bins**

In [91]:

age_bin_encoded = pd.get_dummies(df_encode, columns=['Age_Bin',], dtype=int)

age_bin_encoded.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_Bin_0-18,Age_Bin_18-24,Age_Bin_25-34,Age_Bin_35-50,Age_Bin_51-70,Age_Bin_70+
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,0,0,1,0,1,0,0,0,0,0,1,0
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,0,1,0,0,0,1,0,0,0,0,1,0
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,0,1,0,0,1,0,0,0,1,0,0,0
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,0,0,1,1,0,0,0,0,1,0,0,0
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,1,0,0,0,1,0,0,0,0,1,0,0


In [92]:
age_bin_encoded = age_bin_encoded.drop('Age', axis=1)

## **Creating field Income-to-Amount-Requested **

In [93]:
age_bin_encoded['Income-to-Amount-Requested'] =  age_bin_encoded['Income']/ age_bin_encoded['Amount_Requested']
age_bin_encoded.head()

,Applicant_ID,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_Bin_0-18,Age_Bin_18-24,Age_Bin_25-34,Age_Bin_35-50,Age_Bin_51-70,Age_Bin_70+,Income-to-Amount-Requested
0,217,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,0,0,1,0,1,0,0,0,0,0,1,0,4.935967
1,226,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,0,1,0,0,0,1,0,0,0,0,1,0,0.0
2,240,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,0,1,0,0,1,0,0,0,1,0,0,0,7.486879
3,252,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,0,0,1,1,0,0,0,0,1,0,0,0,9.683497
4,266,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,1,0,0,0,1,0,0,0,0,1,0,0,19.333046


## **Creating field Time_Since_Previous_Assistance**

In [94]:
print(age_bin_encoded['Previous_Assistance_Date'].dtype)

dbdate


In [95]:
age_bin_encoded[age_bin_encoded['Previous_Assistance_Date'].notna()].head()

,Applicant_ID,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_Bin_0-18,Age_Bin_18-24,Age_Bin_25-34,Age_Bin_35-50,Age_Bin_51-70,Age_Bin_70+,Income-to-Amount-Requested
2543,48817,0,3,4865,True,2022-09-04,True,1,19.29.225.228,2024-01-13,0,0,0,1,0,1,0,0,0,0,1,0,0,0.0
2544,29735,35830,1,7587,True,2022-09-13,False,1,33.224.121.169,2024-01-26,0,1,0,0,0,1,0,0,0,1,0,0,0,4.722552
2545,42837,0,1,7515,True,2022-09-13,False,1,26.131.80.176,2024-01-08,0,0,1,0,0,0,1,0,0,0,1,0,0,0.0
2546,14435,29220,1,3620,True,2022-09-14,True,1,207.235.51.152,2024-01-13,0,0,0,1,1,0,0,0,0,0,1,0,0,8.071823
2547,22517,0,2,6029,True,2022-09-14,False,1,72.207.167.220,2024-01-10,0,0,0,1,1,0,0,0,0,0,0,1,0,0.0


In [96]:

age_bin_encoded['Previous_Assistance_Date'] = pd.to_datetime(age_bin_encoded['Previous_Assistance_Date'])
age_bin_encoded['Application_Date'] = pd.to_datetime(age_bin_encoded['Application_Date'])
age_bin_encoded['Time_Since_Previous_Assistance'] = (age_bin_encoded['Application_Date'] - age_bin_encoded['Previous_Assistance_Date']).dt.days

age_bin_encoded[age_bin_encoded['Previous_Assistance_Date'].notna()].head()

,Applicant_ID,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_Bin_0-18,Age_Bin_18-24,Age_Bin_25-34,Age_Bin_35-50,Age_Bin_51-70,Age_Bin_70+,Income-to-Amount-Requested,Time_Since_Previous_Assistance
2543,48817,0,3,4865,True,2022-09-04,True,1,19.29.225.228,2024-01-13,0,0,0,1,0,1,0,0,0,0,1,0,0,0.0,496.0
2544,29735,35830,1,7587,True,2022-09-13,False,1,33.224.121.169,2024-01-26,0,1,0,0,0,1,0,0,0,1,0,0,0,4.722552,500.0
2545,42837,0,1,7515,True,2022-09-13,False,1,26.131.80.176,2024-01-08,0,0,1,0,0,0,1,0,0,0,1,0,0,0.0,482.0
2546,14435,29220,1,3620,True,2022-09-14,True,1,207.235.51.152,2024-01-13,0,0,0,1,1,0,0,0,0,0,1,0,0,8.071823,486.0
2547,22517,0,2,6029,True,2022-09-14,False,1,72.207.167.220,2024-01-10,0,0,0,1,1,0,0,0,0,0,0,1,0,0.0,483.0


## **Change True/False field to 0s and 1s**

In [97]:
bool_columns = age_bin_encoded.select_dtypes(include=['bool']).columns
print(bool_columns)
for col in bool_columns:
    age_bin_encoded[col] = age_bin_encoded[col].astype(int)

Index(['Previous_Assistance_Received', 'Supporting_Doc_Verified'], dtype='object')


In [98]:
age_bin_encoded[['Previous_Assistance_Received','Supporting_Doc_Verified']].head()

,Previous_Assistance_Received,Supporting_Doc_Verified
0,0,0
1,0,0
2,0,1
3,0,1
4,0,0


## **Save to fraud_training_data**

In [99]:
project_id = 'qwiklabs-gcp-04-14e3a67e50ad'
dataset_id = 'fraud_data_raw'
table_name = 'fraud_training_data'

age_bin_encoded.to_gbq(
    destination_table=f'{dataset_id}.{table_name}',
    project_id=project_id,
    if_exists='replace'
)
print(f"\nData saved to {project_id}.{dataset_id}.{table_name}")


/tmp/ipython-input-4089111498.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  age_bin_encoded.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 3708.49it/s]


Data saved to qwiklabs-gcp-04-14e3a67e50ad.fraud_data_raw.fraud_training_data
